In [1]:
using Evolutionary
using Flux
using Flux: onehot, onecold, logitcrossentropy #, throttle, @epochs
using MLDatasets
using Random
using StableRNGs

In [2]:
using ZipFile, CSV, DataFrames, Random, StatsBase , Plots, Statistics

# read file from zip archive

z = ZipFile.Reader("results.zip")

# identify the right file in zip

# The diabetes dataset I found through kaggle have done split into 2 classifiers and select 50 percents of result from each 
# label. So i think there is more work in my dataset

a_file_in_zip = filter(x->x.name == "diabetes_binary_5050split_health_indicators_BRFSS2015.csv", z.files)[1]

#avoid changing the original files in the zip file. However, the dataset will not change but whatever.

a_copy = CSV.File(a_file_in_zip) |> DataFrame

close(z)

#show the dataset

a_copy

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,1.0,1.0,25.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0
3,0.0,1.0,1.0,1.0,29.0,1.0,0.0,0.0
4,0.0,0.0,0.0,1.0,26.0,1.0,0.0,0.0
5,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0
6,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
7,0.0,1.0,1.0,1.0,36.0,0.0,0.0,0.0
8,0.0,0.0,0.0,1.0,26.0,1.0,0.0,0.0
9,0.0,0.0,0.0,1.0,21.0,0.0,0.0,0.0


In [3]:
# Transfer DataFrame to matrix form

df=a_copy|>Tables.matrix

# Transfer the dataset to 2-classifiers. df_0 represents the result is 0, df_1 represents the result is 1.
# Due to my datasset is binary problems, and each result is 50 percents of the whole dataset. So i didn't add any other pre-actions for dataset. 

df_0 = df[df[:,1] .== 0, :]
df_1 = df[df[:,1] .== 1, :]

35346×22 Matrix{Float64}:
 1.0  1.0  1.0  1.0  30.0  1.0  0.0  1.0  …  30.0  1.0  0.0   9.0  5.0  1.0
 1.0  0.0  0.0  1.0  25.0  1.0  0.0  0.0      0.0  0.0  1.0  13.0  6.0  8.0
 1.0  1.0  1.0  1.0  28.0  0.0  0.0  0.0      0.0  1.0  0.0  11.0  4.0  6.0
 1.0  0.0  0.0  1.0  23.0  1.0  0.0  0.0      0.0  0.0  1.0   7.0  5.0  6.0
 1.0  1.0  0.0  1.0  27.0  0.0  0.0  0.0      0.0  0.0  0.0  13.0  5.0  4.0
 1.0  1.0  1.0  1.0  37.0  1.0  1.0  1.0  …   0.0  1.0  1.0  10.0  6.0  5.0
 1.0  1.0  1.0  1.0  28.0  1.0  0.0  1.0      0.0  0.0  1.0  12.0  2.0  4.0
 1.0  1.0  1.0  1.0  27.0  1.0  0.0  0.0     20.0  1.0  0.0   8.0  4.0  7.0
 1.0  1.0  1.0  1.0  34.0  1.0  1.0  0.0      7.0  1.0  0.0   9.0  5.0  4.0
 1.0  1.0  1.0  1.0  24.0  1.0  0.0  0.0      0.0  0.0  0.0  12.0  3.0  3.0
 1.0  1.0  0.0  1.0  31.0  0.0  0.0  0.0  …   5.0  0.0  0.0  13.0  4.0  4.0
 1.0  1.0  1.0  1.0  33.0  1.0  0.0  0.0     30.0  1.0  0.0  11.0  4.0  2.0
 1.0  1.0  1.0  1.0  27.0  1.0  0.0  0.0     30.0  1.0  0.0  1

In [4]:
# The columns of matrix is 35346(here just using 35000 is the same), using random sub-sequence to select 70 percents of 
# columns as the train data.

sample = randsubseq(1:35000, 0.7)
train_df = vcat(df_0[sample, :], df_1[sample, :])

# Then from the not selected columns (which is 30 percents) to select the test data.

notsample = [i for i in 1:35000 if isempty(searchsorted(sample, i))]
test_df = vcat(df_0[notsample, :], df_1[notsample, :])

21374×22 Matrix{Float64}:
 0.0  0.0  1.0  1.0  25.0  1.0  0.0  0.0  …   2.0  0.0  0.0  11.0  5.0  7.0
 0.0  0.0  0.0  0.0  25.0  1.0  0.0  0.0      0.0  0.0  0.0  11.0  6.0  8.0
 0.0  0.0  0.0  1.0  26.0  1.0  0.0  0.0      0.0  0.0  1.0   7.0  6.0  8.0
 0.0  0.0  0.0  1.0  26.0  1.0  0.0  0.0     15.0  0.0  0.0   7.0  5.0  8.0
 0.0  0.0  0.0  1.0  21.0  0.0  0.0  0.0      0.0  0.0  0.0   8.0  5.0  6.0
 0.0  0.0  0.0  1.0  24.0  0.0  0.0  0.0  …   0.0  0.0  0.0   6.0  6.0  8.0
 0.0  0.0  0.0  1.0  23.0  0.0  0.0  0.0      3.0  0.0  0.0   2.0  5.0  8.0
 0.0  0.0  0.0  1.0  22.0  0.0  0.0  0.0     14.0  0.0  0.0   5.0  6.0  8.0
 0.0  0.0  0.0  1.0  25.0  0.0  0.0  0.0      3.0  0.0  1.0   4.0  6.0  8.0
 0.0  0.0  0.0  1.0  26.0  1.0  0.0  0.0      0.0  0.0  0.0   5.0  6.0  8.0
 0.0  1.0  1.0  1.0  36.0  1.0  0.0  0.0  …   5.0  0.0  0.0   8.0  6.0  8.0
 0.0  0.0  1.0  1.0  30.0  1.0  0.0  0.0      0.0  0.0  1.0   3.0  4.0  3.0
 0.0  0.0  0.0  1.0  23.0  0.0  0.0  0.0      3.0  0.0  1.0   

In [5]:
# Divide the columns into features and result. From my dataset, From 2 to 22 columns are the attributes of whether diabetes or not.

X_train = train_df[:, 2:22]
X_test = test_df[:, 2:22]

y_train = train_df[:, 1]
y_test = test_df[:, 1]

21374-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [6]:
using Distributions

# Transfer the X_train to 1 diminension.

dt = fit(ZScoreTransform, X_train, dims=1)

# Using StateBase package to transfer X_train and X_test(make them standard) to the same formate as dt.

X_train_std = StatsBase.transform(dt, X_train)
X_test_std = StatsBase.transform(dt, X_test)

# Transpose the X_train_std and X_test_std.
    
X_train_std, X_test_std= transpose(X_train_std), transpose(X_test_std)

([0.872736083714709 -1.1457982012316115 … 0.872736083714709 0.872736083714709; 0.9479550747216559 -1.0548806178012746 … 0.9479550747216559 -1.0548806178012746; … ; 0.07995124657570614 -0.8902781809237422 … -0.8902781809237422 -1.8605076084231906; 1.0624065850073117 -0.775574099782753 … -2.154059613375302 -0.775574099782753], [-1.1457982012316115 -1.1457982012316115 … 0.872736083714709 0.872736083714709; 0.9479550747216559 -1.0548806178012746 … -1.0548806178012746 -1.0548806178012746; … ; 0.07995124657570614 1.0501806740751545 … 1.0501806740751545 -0.8902781809237422; 0.6029114138097955 1.0624065850073117 … -1.6945644421777855 -0.775574099782753])

In [7]:
base_model=Chain(Dense(21, 8, sigmoid), Dense(8, 1, sigmoid))

Chain(
  Dense(21, 8, σ),                      # 176 parameters
  Dense(8, 1, σ),                       # 9 parameters
)                   # Total: 4 arrays, 185 parameters, 996 bytes.

In [8]:

using Flux: logitbinarycrossentropy
using BSON: @save
opt = ADAM()
loss(x, y) = logitbinarycrossentropy(vec(base_model(x)), y)
accuracy(x, y) = mean(vec(base_model(x) .> 0.5) .== y)
evalcb1() = @show(loss(X_test_std, y_test))
throttled_cb = Flux.throttle(evalcb1, 0.01)
for i in 1:500
    Flux.train!(loss, Flux.params(base_model), [(X_train_std, y_train)], opt, cb = throttled_cb)
end

loss(X_test_std, y_test) = 0.7028674071437077
loss(X_test_std, y_test) = 0.7016774070577106
loss(X_test_std, y_test) = 0.7005117403857953
loss(X_test_std, y_test) = 0.6993717676088848
loss(X_test_std, y_test) = 0.6982585552278701
loss(X_test_std, y_test) = 0.6971728214912756
loss(X_test_std, y_test) = 0.6961148939996938
loss(X_test_std, y_test) = 0.6950846979836849
loss(X_test_std, y_test) = 0.6940817788908947
loss(X_test_std, y_test) = 0.6931053223916169
loss(X_test_std, y_test) = 0.692154199918912
loss(X_test_std, y_test) = 0.6912270344973549
loss(X_test_std, y_test) = 0.6903222827119988
loss(X_test_std, y_test) = 0.6894383365140333
loss(X_test_std, y_test) = 0.6885736049900791
loss(X_test_std, y_test) = 0.6877265927825519
loss(X_test_std, y_test) = 0.6868959476381901
loss(X_test_std, y_test) = 0.6860804900613215
loss(X_test_std, y_test) = 0.6852792240958887
loss(X_test_std, y_test) = 0.6844913221187204
loss(X_test_std, y_test) = 0.6837161138319667
loss(X_test_std, y_test) = 0.682953

In [9]:
@info "MLP" loss=loss(X_train_std,y_train) accuracy = accuracy(X_train_std,y_train)

┌ Info: MLP
│   loss = 0.639469069343596
│   accuracy = 0.7090651092008391
└ @ Main In[9]:1


In [10]:
fitness(m) = loss(X_train_std,y_train,m)

fitness (generic function with 1 method)

In [11]:
loss(x, y,m) = logitbinarycrossentropy(vec(m(x)), y)
accuracy(x, y,m) = mean(vec(m(x) .> 0.5) .== y)

accuracy (generic function with 2 methods)

In [12]:
fitness(base_model)

0.639469069343596

In [13]:
function average_mlp(m1::T, m2::T; rng::Random.AbstractRNG=Random.default_rng()) where {T <: Chain}
    θ1, re1 = Flux.destructure(m1);
    θ2, re2 = Flux.destructure(m2);
    c1, c2 = AX(θ1,θ2; rng=rng)
    return re1(c1), re2(c2)
end

average_mlp (generic function with 1 method)

In [14]:
average_mlp(base_model, base_model; rng=StableRNG(42))

(Chain(Dense(21, 8, σ), Dense(8, 1, σ)), Chain(Dense(21, 8, σ), Dense(8, 1, σ)))

In [42]:
function gaussian_mlp(σ::Real = 1.0)
    vop = gaussian(σ)
    function mutation(recombinant::T,s::NoStrategy; rng::Random.AbstractRNG=Random.default_rng()) where {T <: Chain}  
        θ, re = Flux.destructure(recombinant)
        return re(convert(Vector{Float64}, vop(θ; rng=rng)))
    end
    return mutation
end

gaussian_mlp (generic function with 2 methods)

In [43]:
gaussian_mlp(0.5)(base_model,NoStrategy(); rng=StableRNG(42))

Chain(
  Dense(21, 8, σ),                      # 176 parameters
  Dense(8, 1, σ),                       # 9 parameters
)                   # Total: 4 arrays, 185 parameters, 1.695 KiB.

In [17]:
function gaussian_mlp(σ::Real = 1.0)
    vop = gaussian(σ)
    function mutation(recombinant::T; rng::Random.AbstractRNG=Random.default_rng()) where {T <: Chain}  
        θ, re = Flux.destructure(recombinant)
        return re(convert(Vector{Float32}, vop(θ; rng=rng)))
    end
    return mutation
end

gaussian_mlp (generic function with 2 methods)

In [18]:
import Evolutionary.initial_population
function initial_population(method::M, individual::Chain;
                            rng::Random.AbstractRNG=Random.default_rng(),
                            kwargs...) where {M<:Evolutionary.AbstractOptimizer}
    θ, re = Flux.destructure(individual);
    [re(randn(rng, length(θ))) for i in 1:Evolutionary.population_size(method)]
end

initial_population (generic function with 8 methods)

In [19]:
import Evolutionary.EvolutionaryObjective
function EvolutionaryObjective(f, x::Chain; eval::Symbol = :serial)
    fval = f(x)
    EvolutionaryObjective{typeof(f),typeof(fval),typeof(x),Val{eval}}(f, fval, deepcopy(x), 0)
end

EvolutionaryObjective

In [20]:
EvolutionaryObjective(fitness, base_model)

EvolutionaryObjective{typeof(fitness), Float64, Chain{Tuple{Dense{typeof(σ), Matrix{Float32}, Vector{Float32}}, Dense{typeof(σ), Matrix{Float32}, Vector{Float32}}}}, Val{:serial}}(fitness, 0.639469069343596, Chain(Dense(21, 8, σ), Dense(8, 1, σ)), 0)

In [21]:
import Base: copy

copy(ch::Chain) = deepcopy(ch)

copy (generic function with 184 methods)

In [22]:
opts = Evolutionary.Options(iterations=200, successive_f_tol=25, rng=StableRNG(42))

                  abstol = Inf
                  reltol = Inf
        successive_f_tol = 25
              iterations = 200
             store_trace = false
              show_trace = false
              show_every = 1
                callback = nothing
              time_limit = NaN
         parallelization = serial
                     rng = StableRNGs.LehmerRNG(state=0x00000000000000000000000000000055)


In [40]:
algo = ES(
        mutation =  gaussian_mlp(),
        recombination = average_mlp,
    )

(1/1+1)-ES

In [33]:
algo = GA(
        selection = rouletteinv,
        mutation =  gaussian_mlp(),
        crossover = average_mlp,
        mutationRate = 0.2,
        crossoverRate = 0.9,
        populationSize = 20,
        ε = 0.03
    )

GA[P=20,x=0.9,μ=0.2,ɛ=0.03]

In [25]:
# algo = ES(
#         mutation =  gaussian_mlp(),
#         recombination = average_mlp,
#     )

In [26]:
# θ, re = Flux.destructure(base_model)

In [27]:
# θ

In [28]:
# re 

In [41]:
res = Evolutionary.optimize(fitness, base_model, algo, opts)

LoadError: MethodError: [0mCannot `convert` an object of type 
[0m  Chain{Tuple{Dense{typeof(σ){},Array{[92mFloat32[39m,2},Array{[92mFloat32[39m,1}},Dense{typeof(σ){},Array{[92mFloat32[39m,2},Array{[92mFloat32[39m,1}}}}[0m to an object of type 
[0m  Chain{Tuple{Dense{typeof(σ){},Array{[91mFloat64[39m,2},Array{[91mFloat64[39m,1}},Dense{typeof(σ){},Array{[91mFloat64[39m,2},Array{[91mFloat64[39m,1}}}}
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::T[39m) where T at F:\Julia-1.7.1\share\julia\base\essentials.jl:218

In [30]:
evomodel= Evolutionary.minimizer(res)
@info "MLP" loss=loss(X_train_std,y_train, evomodel) accuracy = accuracy(X_train_std,y_train, evomodel)

┌ Info: MLP
│   loss = 0.6678961122418255
│   accuracy = 0.5749393328671903
└ @ Main In[30]:2
